In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode(connected=True)
import utils as u

In [3]:
"""
Plain old dataset:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)

Enhanced with impressions:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)
"""

'\nPlain old dataset:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n\nEnhanced with impressions:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n'

In [4]:
"""
These destinations are selected based on their query pool sizes, and how representative
they are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)

9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),
482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),
9D883ED995259EA3E715122397FE0EF9 has query pool of size 2
"""
# selected_destinations = ["9FF55", "482E9", "9D883"]

'\nThese destinations are selected based on their query pool sizes, and how representative\nthey are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)\n\n9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),\n482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),\n9D883ED995259EA3E715122397FE0EF9 has query pool of size 2\n'

In [5]:

# def extra_df_prep(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.assign(
#         destination=df.apply(
#             lambda row: row.destination[0:5],
#             axis=1,
#         )
#     )
#     global selected_destinations
#     return df[df.destination.isin(selected_destinations)]


In [6]:
# unaugmented path
path = "ray/criteo/bias_varying_epoch_size"

In [7]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [8]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [9]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [10]:
# u.save_data(path)

In [11]:
# augmented impressions path
path = "ray/criteo/augmented_bias_varying_epoch_size"

In [12]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [13]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [14]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [15]:
# augmented convesrions path
path = "ray/criteo/augmented_conversions_bias_varying_epoch_size"

In [16]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [17]:
u.save_data(path)

In [18]:
df = pd.read_csv("../data/criteo/criteo_query_pool_augmented_conversions.csv")
df.head()

,partner_id,user_id,filter,product_category3,count,conversion_timestamp,epsilon,aggregatable_cap_value,key
0,BC24F033D166444EEB6CC79BF6213A72,AE0C6BD2600A4792BFC001C09F88B958,NaN,D3915F52523CFCDC7FA05E5E96774B89,5.0,1596439471,0.511686,5,51
1,6CCBF9BF3B87306C95BD5CD15F4CD324,DE9FA9092CB34644AFCB2F6EFE60EE58,NaN,CBEF3E31D31D28182362A050C392548F,3.0,1596439473,0.511686,5,90
2,6BFFDE668C5BC3938C875D3D0940E699,87E3C38C7A7E4313924ECE5ACF1C1E3A,NaN,30131E72F860C21850E0F2BB16B862DF,2.0,1596439474,0.511686,5,350
3,35EA60602589E5F5F00520A563290A59,A7B2BB8715A5458285E7129B158295C3,NaN,755FCFBCC31578C16D48289ED6830BF5,3.0,1596439478,0.617315,5,4
4,43B9B57374688E5B843DC51BF98F1ED7,9481F74666CF4C5DBF718351DAFF09E6,NaN,3AAA3CBC1A566F6A8FD4614DED1AE7AB,3.0,1596439481,0.511686,5,0


In [19]:
true_output = df.groupby(["partner_id", "key"])["count"].sum()

In [20]:
print(true_output.shape, true_output[true_output > 0].shape)

(7444,) (7444,)
